In [1]:
#import library
import cv2
from torchvision import transforms, models, datasets
import torch.nn as nn
import torch
import numpy as np
import matplotlib.pyplot as plt
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [2]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

In [3]:
#define vgg16 model
def get_model():
    model = models.vgg19(pretrained = True)
    # Freeze parameters so we don't backprop through them
    for param in model.parameters():
        param.requires_grad = False
    model.avgpool = nn.Sequential(
        nn.Conv2d(512,512, kernel_size=3),
        nn.MaxPool2d(2),
        nn.ReLU(),
        nn.Flatten()
    )
    class ageGenderClassifier(nn.Module):
        def __init__(self):
            super(ageGenderClassifier, self).__init__()
            self.intermediate = nn.Sequential(
                nn.Linear(2048,512),
                nn.ReLU(),
                nn.Dropout(0.4),
                nn.Linear(512,128),
                nn.ReLU(),
                nn.Dropout(0.4),
                nn.Linear(128,64),
                nn.ReLU(),
            )
            self.age_classifier = nn.Sequential(
                nn.Linear(64, 1),
                nn.Sigmoid()
            )
            self.gender_classifier = nn.Sequential(
                nn.Linear(64, 1),
                nn.Sigmoid()
            )
        def forward(self, x):
            x = self.intermediate(x)
            age = self.age_classifier(x)
            gender = self.gender_classifier(x)
            return gender, age

    model.classifier = ageGenderClassifier()

    return model.to(device)


In [4]:
def model_prediction(pre_img, model):

    face = pre_img.to(device).float()
    gender, age = model(face)

    pred_gender = gender.to('cpu').detach().numpy()
    pred_age = age.to('cpu').detach().numpy()

    return pred_gender, pred_age

In [5]:
from google.colab import drive
!mkdir -p /content/drive/MyDrive/vgg19 # Create the directory if it doesn't exist
drive.mount('/content/drive/MyDrive/vgg19') # Remove the space in the path

Mounted at /content/drive/MyDrive/vgg19


In [6]:
device = torch.device('cpu')  # Use the GPU

pretrained_weights = torch.load('/content/drive/MyDrive/vgg19/MyDrive/vgg19', map_location=device)
model = get_model()
model.load_state_dict(pretrained_weights, strict = False)

model.eval()

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth
100%|██████████| 548M/548M [00:02<00:00, 217MB/s]


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padd

In [7]:
import torch

def detection(gray):

    model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
    # model = torch.hub.load('ultralytics/yolov5', model_name, source=model_url)
    result = model(gray)

    face = result.pandas().xyxy[0]
    return face

In [16]:
def preprocess_image(face, frame):
    # Iterate over the rows of the DataFrame and extract numerical values
    for index, row in face.iterrows():
        frame = np.array(frame)

        x1, y1, x2, y2 = int(row['xmin']), int(row['ymin']), int(row['xmax']), int(row['ymax'])
        x1, y1, x2, y2 = int(x1+(1/3*x1)), int(y1+(1/3*y1)), int(x2-(1/5*x2)), int(y2-(1/5*y2))

        crop_frame = frame[y1:y2, x1:x2]
        crop_frame = cv2.resize(crop_frame, (224, 224))

        crop_frame = torch.tensor(crop_frame).permute(2,0,1)
        crop_frame = normalize(crop_frame/255.)

        return crop_frame[None]

In [19]:
cap = cv2.VideoCapture('/content/face3.mp4')

print(cap.isOpened())

True


In [ ]:
#Capture video from webcam
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_size = (width, height)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')

# Initialize video writer.
video_output = cv2.VideoWriter('output.mp4', fourcc, fps, frame_size)


age_list = []
while cap.isOpened():

    success, frame = cap.read()

    if not success:
        print("Null.Frames")
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    detect_face = detection(gray)

    pre_img = preprocess_image(detect_face, frame)
    gender, age = model_prediction(pre_img, model)

    gender = np.where(gender[0][0]<0.5,'Male','Female')
    age = int(age[0][0]*116)
    age_list.append(age)
    print(age)


    for _, row in detect_face.iterrows():
        x1, y1, x2, y2 = int(row['xmin']), int(row['ymin']), int(row['xmax']), int(row['ymax'])

        x1, y1, x2, y2 = int(x1+(1/4*x1)), int(y1+(1/4*y1)), int(x2-(1/5*x2)), int(y2-(1/5*y2))
        cv2.rectangle(frame, (x1,y1), (x2, y2), (100, 50, 200), 3)

        cv2.putText(frame, f'Gender: {gender}, Age: {max(age_list)}', (x1, y2), cv2.FONT_HERSHEY_COMPLEX_SMALL, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
    #    video_output.write(image)

    video_output.write(frame)

cap.release()
video_output.release()